<font color='green'> 
**Youtube - Aladdin Persson Kanalı - TensorFlow 2.0 Beginner Tutorials serisi**
    
TensorFlow Tutorial 9 - Custom Layers - Aladdin Persson anlattı.
</font>

**Video**: [TensorFlow Tutorial 9 - Custom Layers](https://www.youtube.com/watch?v=cKMJDkWSDnY&list=PLhhyoLH6IjfxVOdVC1P1L5z5azs0XjMsb&index=9)

### İçindekiler

**Loading Dataset**

**Preprocessing Dataset**

**Creating a Basic Neural Network**

**Creating Custom Layers - 1**
* We created a class for our layers by initializing weight and bias ourselves and integrated it into our model.

**Creating Custom Layers - 2**
* When we were going to integrate the class we created into our model, we had to give the input size ourselves. In order not to do this, we added a function called *build()* to our class.

**Creating Custom Layers - 3**
* We created also a class for running the relu function.


### <font color="blue"> Giriş</font>

Nasıl custom layer oluşturduğumuza bakacağız.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### 1. Loading Dataset

In [2]:
from tensorflow.keras.datasets import mnist

In [3]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


### 2. Preprocessing Dataset

In [5]:
x_train = x_train.reshape(-1, 28*28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28*28).astype("float32") /255.0

### 3. Creating a Basic Neural Network


##### <font color="green"> `class MyModel(keras.Model)` ile Neural Network oluşturuyoruz. </font>

Convolutional layerları batch norm ve relu ile defalarca tekrarlayabildiğimiz bir class yazmıştık önceki notebookta. Bu classta flatten yapıp dense layer eklemiştik. Burada direkt bu classı yazıyoruz. Normal neural network oluşturuyoruz CNN değil dolayısıyla conv-batch norm-relu yapsın diye yazdığımız classa ihtiyacımız yok.

In [6]:
class MyModel(keras.Model): 
  def __init__(self, num_classes=10):
    super(MyModel, self).__init__()
    self.dense1 = layers.Dense(64)
    self.dense2 = layers.Dense(num_classes)

  def call(self, input_tensor):
    x = tf.nn.relu(self.dense1(input_tensor))
    return self.dense2(x)

In [7]:
model = MyModel()

##### <font color="green"> `model.compile()` içerisinde ağımızın eğitim bölümünü nasıl yapılandıracağımızı anlatıyoruz kerasa. </font>

In [8]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics=["accuracy"],
)

##### <font color="green"> `model.fit()` ile yapılandırdığımız modeli eğitiyoruz. </font>

In [9]:
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)

Epoch 1/2
1875/1875 - 6s - loss: 0.3058 - accuracy: 0.9127
Epoch 2/2
1875/1875 - 3s - loss: 0.1465 - accuracy: 0.9577


##### <font color="green"> `model.evaluate()` ile test setimizi gönderip modelimizin başarısını değerlendiriyoruz. </font>

In [10]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 1s - loss: 0.1215 - accuracy: 0.9626


[0.12154971063137054, 0.9625999927520752]

Yukarıda modelimizi oluştururken `layers.Dense(64)` koduyla kerasın içinde yer alan layers'ı ve onun içinde yer alan Dense layersı kullandık. Ayrıca yine `tf.nn.relu(self.dense1(input_tensor))` koduyla tensorflowun relu metodunu kullandık. Fakat kendimiz de bunları oluşturabiliriz. Bunu nasıl yapacağımıza bakacağız.

### 4. Creating Custom Layers - 1

In [11]:
class Dense(layers.Layer):
  def __init__(self, units, input_dim): # burada input dimensionını belirtiyoruz.
    super(Dense,self).__init__()
    self.w = self.add_weight(
        name = 'w', 
        shape = (input_dim, units), # input_dim = 784 units = 64
        initializer = 'random_normal',
        trainable = True,
    )
    self.b = self.add_weight(
        name = 'b', shape =(units,), initializer='zeros', trainable=True,
    )

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

* `name = 'w'`: bir sonraki videoda modeli save ve load edeceğimizi göreceğiz. İsim belirtmezsek kaydedemiyoruz modeli. 

* `shape = (input_dim, units)` : input_dim = 784, units ise bunu mapleyeceğimiz nod sayısı. Yukarıda `self.dense1 = layers.Dense(64)` yazmıştık. Buradaki units de 64 olacak.

*  trainable = True, batch normda vs. parametreler trainable değil. Ama bu dense layerdaki tüm parametrelerimiz için trainable olsun dedik.

* Yukarıda x_train ve x_testin boyutları (60000, 28, 28) idi. Bunları preprocessing aşamasında `x_train = x_train.reshape(-1, 28*28).astype("float32") / 255.0` ve `x_test = x_test.reshape(-1, 28*28).astype("float32") / 255.0` koduyla (60000, 784) haline getirdik. Burada w'nin boyutuna da (input_dim=784, units=64) demiştik. x ve w'lerin çarpımı sonucu tek bir resim için (60000 için değil) elimizde (1,64) boyutunda bir matris oluşuyor. (64,) boyutunda bir bias eklediğimizde boyut (1,64) oluyor.

###### <font color="purple">`x*w + b`'daki broadcastingle ilgili kendi çalışmam</font>

In [32]:
import numpy as np

d = [4,3,2,45,2,25,24,6]

In [33]:
k= np.reshape(d,(1,8))

In [34]:
l= np.reshape(d,(8,))

In [35]:
print(k)
print(l)

[[ 4  3  2 45  2 25 24  6]]
[ 4  3  2 45  2 25 24  6]


In [37]:
print(k.shape)
print(l.shape)

(1, 8)
(8,)


In [36]:
k+l

array([[ 8,  6,  4, 90,  4, 50, 48, 12]])

In [38]:
(k+l).shape

(1, 8)

###### Devam

Kendi yazdığımız Dense layerları ekliyoruz classımıza.

In [12]:
class MyModel(keras.Model):
  def __init__(self, num_classes=10):
    super(MyModel,self).__init__()
    self.dense1 = Dense(64, 784)
    self.dense2 = Dense(10,64)

  def call(self, input_tensor):
    x = tf.nn.relu(self.dense1(input_tensor))
    return self.dense2(x)

In [13]:
model = MyModel()

##### <font color="green"> `model.compile()` içerisinde ağımızın eğitim bölümünü nasıl yapılandıracağımızı anlatıyoruz kerasa. </font>

In [14]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics=["accuracy"],
)

##### <font color="green"> `model.fit()` ile yapılandırdığımız modeli eğitiyoruz. </font>

In [15]:
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)

Epoch 1/2
1875/1875 - 3s - loss: 0.3385 - accuracy: 0.9076
Epoch 2/2
1875/1875 - 3s - loss: 0.1641 - accuracy: 0.9529


##### <font color="green"> `model.evaluate()` ile test setimizi gönderip modelimizin başarısını değerlendiriyoruz. </font>

In [16]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 1s - loss: 0.1434 - accuracy: 0.9569


[0.14339935779571533, 0.9569000005722046]

Neredeyse aynı accuracyi verdi önceki yazdığımız modelle.

### 5. Creating Custom Layers - 2 

* Kerasla yazdığımız layerlarda input dimensionını belirtmemiştik sadece output dimensionları yazıyorduk. 

    `self.dense1 = layers.Dense(64)`

    `self.dense2 = layers.Dense(num_classes)`

* Kendi yazdığımız custom layerlarda ise belirttik.

    `self.dense1 = Dense(64, 784)`

    `self.dense2 = Dense(10,64)`

Biz de input dimensionını kaldırabiliriz. Kolaylık olsun diye belirtmek istemeyebiliriz. Bunu yapmak için:
* `Dense(layers.Layer)` classında `def __init__(self, units, input_dim)` kısmında input_dim diye belirtmiştik, bunu kaldırdık. 

* Bu kısımda tanımladığımız weight ve biasları `def build(self, input_shape)` fonksiyonu oluşturup burada tanımladık. 

* `shape = (input_dim, units)` içerisinde belirttiğimiz input_dim yerine `input_shape[-1]` yazdık. Bizim inputumuz (60000, 784) shape'indeydi. Dolayısıyla 784'ü alacak.

* units'i ise `__init__` fonksiyonunda `self.units = units` ile kullanıcıdan alıyoruz ve `shape = (input_shape[-1], units)`daki units yerine `self.units` diyoruz. Aynı şekilde self.b'deki `shape =(units,)`deki unitsi de `self.units` ile değiştiriyoruz.

Bunları yaptığımızda MyModel'i oluştururken dense layerlarda input dimensionını belirtmemize gerek kalmıyor.

Tensorflow'un tutorialındaki [şu olayı](https://www.tensorflow.org/guide/keras/custom_layers_and_models) uygulamış. 


In [17]:
class Dense(layers.Layer):
  def __init__(self, units):
    super(Dense,self).__init__()
    self.units = units
    
  def build(self, input_shape): 
    self.w = self.add_weight(
        name = 'w', 
        shape = (input_shape[-1], self.units), # input_shape kerasa ait bir argüman
        initializer = 'random_normal',
        trainable = True,
    )
    self.b = self.add_weight(
        name = 'b', shape =(self.units,), initializer='zeros', trainable=True,
    )

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

[build() metoduyla ilgili tensorflow'un açıklaması](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer)
> build(self, input_shape): This method can be used to create weights that depend on the shape(s) of the input(s), using add_weight(). __call__() will automatically build the layer (if it has not been built yet) by calling build().



In [18]:
class MyModel(keras.Model):
  def __init__(self, num_classes=10):
    super(MyModel,self).__init__()
    self.dense1 = Dense(64)
    self.dense2 = Dense(num_classes)

  def call(self, input_tensor):
    x = tf.nn.relu(self.dense1(input_tensor))
    return self.dense2(x)

In [19]:
model = MyModel()

##### <font color="green"> `model.compile()` içerisinde ağımızın eğitim bölümünü nasıl yapılandıracağımızı anlatıyoruz kerasa. </font>

In [20]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics=["accuracy"],
)

##### <font color="green"> `model.fit()` ile yapılandırdığımız modeli eğitiyoruz. </font>

In [21]:
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)

Epoch 1/2
1875/1875 - 3s - loss: 0.3457 - accuracy: 0.9059
Epoch 2/2
1875/1875 - 3s - loss: 0.1614 - accuracy: 0.9536


##### <font color="green"> `model.evaluate()` ile test setimizi gönderip modelimizin başarısını değerlendiriyoruz. </font>

In [22]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 1s - loss: 0.1314 - accuracy: 0.9607


[0.13139474391937256, 0.9606999754905701]

Bu şekilde MyModel'in içine yazdığımız Dense layer kodlarını kerasın kodlarına fazlasıyla benzetmiş olduk.



### 6. Creating Custom Layers - 3 

`x = tf.nn.relu(self.dense1(input_tensor))` yerine kendimiz relu fonksiyonu yazmak istersek nasıl yazacağımıza bakacağız. Bunun için `MyReLu(layers.Layer)` classı oluşturuyoruz ve bunu MyModel classımızda initializer kısmına ekliyoruz. Class yerine fonksiyon olarak da tanımlayabilirdik. 

In [23]:
class Dense(layers.Layer):
  def __init__(self, units):
    super(Dense,self).__init__()
    self.units = units
    
  def build(self, input_shape): 
    self.w = self.add_weight(
        name = 'w', 
        shape = (input_shape[-1], self.units), # input_shape kerasa ait bir argüman
        initializer = 'random_normal',
        trainable = True,
    )
    self.b = self.add_weight(
        name = 'b', shape =(self.units,), initializer='zeros', trainable=True,
    )

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

In [24]:
class MyReLU(layers.Layer):
  def __init__(self):
    super(MyReLU,self).__init__()

  def call(self,x):
    return tf.math.maximum(x,0) # Hangi değer maksimumsa o değeri döndürecek (relunun çalışma mantığı)

In [25]:
class MyModel(keras.Model):
  def __init__(self, num_classes=10):
    super(MyModel,self).__init__()
    self.dense1 = Dense(64)
    self.dense2 = Dense(num_classes)
    self.relu = MyReLU() # MyReLU classını örneklendiriyoruz böylelikle. Class yerine fonksiyon olarak tanımlasaydık buraya eklemeyecektik.

  def call(self, input_tensor):
    x = self.relu((self.dense1(input_tensor)))
    return self.dense2(x)

`x = tf.nn.relu(self.dense1(input_tensor))` yerine `x = self.relu((self.dense1(input_tensor)))` yazdık call kısmında.

In [26]:
model = MyModel()

##### <font color="green"> `model.compile()` içerisinde ağımızın eğitim bölümünü nasıl yapılandıracağımızı anlatıyoruz kerasa. </font>

In [27]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics=["accuracy"],
)

##### <font color="green"> `model.fit()` ile yapılandırdığımız modeli eğitiyoruz. </font>

In [28]:
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)

Epoch 1/2
1875/1875 - 3s - loss: 0.3506 - accuracy: 0.9042
Epoch 2/2
1875/1875 - 3s - loss: 0.1656 - accuracy: 0.9520


##### <font color="green"> `model.evaluate()` ile test setimizi gönderip modelimizin başarısını değerlendiriyoruz. </font>

In [29]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 1s - loss: 0.1302 - accuracy: 0.9622


[0.1302354484796524, 0.9621999859809875]